## Problem Statement 6

### Implement the Continuous Bag of Words (CBOW) Model for the given (textual document 1) using the below steps:
    a. Data preparation
    b. Generate training data
    c. Train model
    d. Output

### Import necessary packages

In [5]:
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

from sklearn.metrics.pairwise import euclidean_distances

import numpy as np
import pandas as pd

In [6]:
data = """Deep learning (also known as deep structured learning) is part of a
broader family of machine learning methods based on artificial neural networks
with representation learning. Learning can be supervised, semi-supervised or unsupervised.
Deep-learning architectures such as deep neural networks, deep belief networks,
deep reinforcement learning, recurrent neural networks, convolutional neural networks and
Transformers have been applied to fields including computer vision, speech recognition,
natural language processing, machine translation, bioinformatics, drug design,
medical image analysis, climate science, material inspection and board game programs,
where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

### a. Data preparation

In [7]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(dl_data)

words2id=tokenizer.word_index
words2id['PAD']=0

id2words={v:k for k,v in words2id.items()}

wids=[[words2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size=len(words2id)
embed_size=100
window_size=2

print("Vocabulary size: ", vocab_size)
print("Vocabulary Sample: ", list(words2id.items())[:10])

Vocabulary size:  75
Vocabulary Sample:  [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


### b. Generate training data

In [8]:
def generate_context_word_pair(corpus, window_size, vocab_size):
    context_length=window_size*2

    for words in corpus:
        sentence_length = len(words)

        for index, word in enumerate(words):
            context_words = []
            label_word = []
            start = index - window_size
            end = index + window_size + 1

            context_words.append([words[i]
                                 for i in range(start, end)
                                 if 0 <=i <sentence_length
                                 and i!= index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen = context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x,y)



### c. Train Model

In [9]:
cbow=Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x:K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation="softmax"))


cbow.compile(loss="categorical_crossentropy", optimizer="rmsprop")
print(cbow.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
for epoch in range(1,6):
    loss=0
    i=0

    for x,y in generate_context_word_pair(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i+=1
        loss+=cbow.train_on_batch(x,y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:',epoch, '\tLoss:', loss)
    print()



Epoch: 1 	Loss: 434.23626804351807

Epoch: 2 	Loss: 429.23753786087036

Epoch: 3 	Loss: 425.6741223335266

Epoch: 4 	Loss: 422.3146245479584

Epoch: 5 	Loss: 419.79110407829285



In [11]:
weights=cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2words.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.015630,0.060666,-0.006925,0.012304,0.007216,-0.009098,0.012355,0.061362,0.031274,-0.050026,...,-0.009281,-0.010069,-0.028608,0.029703,0.056208,-0.017927,-0.026966,0.010226,0.054954,-0.031233
networks,0.045897,-0.013697,0.031511,0.027852,-0.035291,-0.007903,0.004157,0.015570,0.065559,0.026669,...,-0.045831,-0.015098,-0.055646,-0.007218,0.006770,-0.003054,-0.027773,-0.012163,0.027630,-0.062146
neural,0.029271,-0.032125,-0.042396,-0.035244,-0.046477,-0.014111,0.002366,-0.000009,-0.024210,-0.000592,...,-0.010499,0.027931,0.031971,0.049539,-0.044071,0.006587,0.024373,0.030428,0.008566,-0.008183
and,-0.016201,0.036867,0.005533,0.043987,0.026676,-0.037311,-0.039743,-0.016887,-0.003169,0.024882,...,-0.040443,-0.033773,0.034780,-0.033097,0.041941,-0.000294,0.007498,0.001366,0.048161,-0.004407
as,0.026542,-0.017806,-0.037797,-0.042589,-0.006078,0.014452,0.004903,-0.020666,-0.005896,0.035851,...,0.036520,0.018890,0.000166,0.045780,-0.030948,0.028548,0.038360,0.009921,-0.009759,0.041904


### d. Output

In [ ]:
# It asks for the input

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

inwords = input()

similar_words = {search_term: [id2words[idx] for idx in distance_matrix[words2id[search_term]-1].argsort()[0:6]]
                   for search_term in {inwords}}

similar_words

(74, 74)


{'convolutional': ['recurrent',
  'recognition',
  'deep',
  'produced',
  'neural',
  'been']}